# Inaugural Project

Imports and set magics:

In [ ]:
from types import SimpleNamespace
import warnings 
import numpy as np
from scipy import optimize
import pandas as pd 
import matplotlib.pyplot as plt
import math
import plotly.graph_objects as go
from tabulate import tabulate

# Autoreload modules when code is run. Otherwise, python will not see recent changes. 
%load_ext autoreload 
%autoreload 2

# Import inauguralproject
from inauguralproject import HouseholdSpecializationModelClass

# Ignore RuntimeWarning
warnings.filterwarnings("ignore", category=RuntimeWarning)

model = HouseholdSpecializationModelClass()

# Question 1

First, assuming that the choice set is discrete meaning that $L_M, L_F, H_M$ and $H_F$ are available in 
half hours. To illustrate how $H_F/H_M$ changes for varying $\alpha$ and $\sigma$ values we first defines the arrays for the two parameters. We then create three empty lists are created to store the results of the loop. The loop iterates over all possible combinations of $\alpha$ and $\sigma$ values, with the parameters being set to the current values of $\alpha$ and $\sigma$ in each iteration. For the current parameter values the model is solved and the values of $H_F$ and $H_M$ is calculated as the the ratio and stored in the list. Tabulate is used to present results in a table, which can be found below. 

In [ ]:
# Define alpha and sigma values to vary
alpha_vec = np.linspace(0.25, 0.75, 3)
sigma_vec = np.linspace(0.50, 1.50, 3)

# Create empty lists to store results
alpha_list = []
sigma_list = []
HFHM_ratio_list = []

# Loop over alpha and sigma values
for alpha in alpha_vec:
    for sigma in sigma_vec:
        # Set new alpha and sigma values
        model.par.alpha = alpha
        model.par.sigma = sigma
        # Solve model
        opt = model.solve_discrete()
        # Store results
        alpha_list.append(alpha)
        sigma_list.append(sigma)
        HFHM_ratio_list.append(opt.HF/opt.HM)

# Create DataFrame with results
results_df = pd.DataFrame({'Alpha': alpha_list,
                           'Sigma': sigma_list,
                           'Home Production Ratio': HFHM_ratio_list})

# Convert dataframe to tabular format
table = tabulate(results_df,headers='keys', 
                            tablefmt='fancy_grid')

# Print the table
print(table)

We see from the table that for $\alpha = 0.50$ the ratio equals $1$ as the male and the female are equally productive in home production. An $\alpha$ value closer to $0.00$ indicates that the female specializes more in home production, while the male specializes more in market work. Opposite an $\alpha$ value closer to $1.00$ indicates a female specialization in market work and male specialization in home production. 

$\sigma$ represent the elasticity of substitution such that $\sigma$ equals 1 means that the time and effort for the female and male memeber of the household is perfect substitutes. Meaning that the marginal productivity of one household member's work is constant regardless of the other household member's time allocation. For $\sigma$ less than $1.00$, the household members' time and effort are substitutes, which means that the productivity of one household member's work decreases when the other household member also works. When $\sigma$ is greater than $1.00$, the household members' time and effort are complements, which means that the productivity of one household member's work increases when the other household member also works.

The home production ratio for varying $\alpha$ and $\sigma$ values is illustrated in the 3D plot below.
To create this plot, we first use the values of $\alpha$, $\sigma$, and the home production ratio from 'results_df', as defined above, to determine the x-axis, y-axis, and z-axis values, respectively. Next, we update the layout of the plot by setting the titles for the axes. To add hover labels, we use 'update_traces' and specify the format of the labels, including the values of $\alpha$, $\sigma$, and the home production ratio. 

In [ ]:
# Create 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=results_df['Alpha'],
    y=results_df['Sigma'],
    z=results_df['Home Production Ratio'],
    text=results_df['Home Production Ratio'],
    mode='markers',
    marker=dict(
        size=8,
        color=results_df['Home Production Ratio'],
        colorscale='plasma',
        opacity=0.8
    )
)])

# Update layout
fig.update_layout(
    scene=dict(
        xaxis_title='Alpha',
        yaxis_title='Sigma',
        zaxis_title='Home Production Ratio'
    ),
    title='Home Production Ratio vs Alpha and Sigma',
)

# Add hover labels
fig.update_traces(hovertemplate='Alpha: %{x:.2f}<br>Sigma: %{y:.2f}<br>Home Production Ratio: %{z:.2f}')

# Show the 3D plot
fig.show()

The graph shows that the we reach the highest values of home production ratio when $\alpha$ equals 0.75 and $\sigma$ equals 1.50. 

# Question 2

We assume the the values of $\alpha$ and $\sigma$ are back to the baseline values, $\alpha=0.5$ and $\sigma=1.0$. We define female wage, $w_f$, to vary between 0.8 and 1.2. As in question $1$ we also create empty lists for $log(H_F/H_M)$ to store the results later in loop. In each iteration of the loop $w_f$ is set to a current value and the model is solved using the solve_discrete function. The logarithmic values of $w_F/w_M$ and $H_F/H_M$ are appended to the lists for $log(w_F/w_M)$ and $log(H_F/H_M)$. 

To plot the results we use 'go.Figure' where the plot is defines as a scatter object. The x-axis is set to $log(w_F/w_M)$ and the y-axis is set to $log(H_F/H_M)$. The markers shows the varying values of $w_f$. The figure is shown by running the code below. 

In [ ]:
# Reset values for alpha and sigma back to initial values
model.par.alpha = 0.5
model.par.sigma = 1.0

# Define wf to vary and create empty lists
wf_vec = np.linspace(0.8, 1.2, 5)
log_wfwm_list = []
log_HFHM_ratio_list = []

# Loop over the female wage values
for wf in wf_vec:
    model.par.wF = wf
    opt = model.solve_discrete()
    log_wfwm_list.append(math.log(wf / model.par.wM))
    log_HFHM_ratio_list.append(math.log(opt.HF / opt.HM))

# Create figure with results
fig = go.Figure(data=go.Scatter(x=log_wfwm_list,
                                y=log_HFHM_ratio_list,
                                mode='lines+markers',
                                marker=dict(color='blue'),
                                line=dict(width=2)
))

# Update layout
fig.update_layout(
    xaxis=dict(
        title='log(wF/wM)',
        tickformat=".2f"
    ),
    yaxis=dict(
        title='log(HF/HM)',
        tickformat=".2f"
    ),
    title='Discrete time: Variation in Wage Ratio and Ratio of Household Production Time',
)

# Show the interactive line plot
fig.show()

The figure shows that the higher value of female wage, the lower value for $log(H_F/H_M)$, meaning the allocation of time changes such the female member works more hours at home than the male household member. When the female wage increases further, the female is better off working in the labour market than home production compared to the male member.

# Question 3

We now assume that the choice set is continous and define a method 'Solve' that solves the model continuously, meaning that it finds the optimal combination of hours worked by each member of the household at any given time. 
First we defines an objective function which takes 4 variables as input, $L_F$, $L_M$, $H_F$ and $H_M$ and then defines constraints, bounds and initial values. 

To solve the model continuously we use the optimization techniques from the 'scipy.optimize' package to find the optimal values of hours. The optimal combination of hours worked that maximizes the utility is found and stored in the opt. namespace.

From question 2 we use the definition of $w_f$ and the same approach to plot $log(H_M/H_F)$ against $log(w_m/w_f)$. By running the code below, you will see the plot. 

In [ ]:
# Define empty lists for log(HF/HM) and log(wF/wM)
log_HFHM_list = []
log_wfwm_list = []

# Loop over the female wage values
for wF in model.par.wF_vec:
    model.par.wF = wF
    opt = model.solve_continuous()
    log_HFHM = np.log(opt.HF / opt.HM)
    log_HFHM_list.append(log_HFHM)
    log_wfwm = np.log(model.par.wF / model.par.wM) 
    log_wfwm_list.append(log_wfwm)

# Create figure with results
fig = go.Figure(data=go.Scatter(x=log_wfwm_list,
                                y=log_HFHM_list,
                                mode='lines+markers',
                                marker=dict(color='green'),
))

# Update layout
fig.update_layout(
    xaxis=dict( title='log(wF/wM)',
                tickformat=".2f"),
    yaxis=dict( title='log(HF/HM)',
                tickformat=".2f"),
    title='Continuous time: Variation in Wage Ratio and Ratio of Household Production Time',
)

# Show the interactive plot
fig.show()

As in question 2, we see from the figure above that the higher value of the female wage, the lower value of $log(H_F/H_M)$.

# Question 4

To match the data from *Siminski and Yetsenga (2004)* we have to choose values of $\alpha$ and $\sigma$ to minimize

$$
(\beta_0 - \hat\beta_0)^2 + (\beta_1 - \hat\beta_1)^2
$$

To solve the model we use the continuous values for $L_M, L_F, w_M$ and $w_F$. 

First we define a method to estimate the parameters $\alpha$ and $\sigma$ to minimize an objective function. The method solves the model for all $w_f$ values and runs a regression defined in 'run_regression'. Then we defines the objective function to be minimizes, which returns the difference between the target values of $\beta_0$ and $\beta_1$ and the estimated values. Next, we create bounds and initial values which is used to call the optimizer to minimize the objective function using the SLSQP method. Finally, it stores the results in a dictionary and the code below prints them out.

In [ ]:
# Print the rsults 
model.estimate()

print(f'The squared erroes are minimized for the optimal values are alpha = {model.sol.alpha:.3f} and sigma = {model.sol.sigma:.3f}')

By the regression equaltion $log(H_F/H_M) = \beta_0 + \beta_1 * log(w_f/w_m)$ we see that $\beta_0$ is a constant term and $\beta_1$ is the coefficient for the wage ratio for the female and male household member. When the wage is equal for men and women, we have that $w_F / w_m = 1$ which implies that $log(H_F/H_M) = 0.4$ which means that women spend more hours doing work in the household compared to men. 

Interpretation of $\beta_1$ means that change in the wage ratio leads to a $\beta_1$ percent change in the ratio of home production. Siminiske and Yetsenga (2022) find that $\beta_1$ equals $-0.100$ meaning that a 1 percent change in the the wage ratio will decrease the ratio of home production with 0.1 pct.

We estimate that $\sigma$ equal 0.100 which indicates that the household members' time and effort are substitutes. It means that the household members are relatively more productive in the market work than in household production. The household is more likely to specialize in market work rather than home production.

We estimate $\alpha$ equals 0.98 which is close to 1.00. It indicates that the male householde member are more productive working in the market and the female household member is relatively more productive in home production. 
Overall this means that the male household member will take almost all of the work in the market while the female household member takes in almost all the home production.

The plot above shows the values of the objective function for different combinations of the parameters alpha and sigma. The height of the surface corresponds to the value of the objective function. The lowest point is where alpha = 0.99 and sigma = 0.10.

In [ ]:
# Define alpha and sigma values to loop over
alpha_vals = np.linspace(0.90, 0.99, 10)
sigma_vals = np.linspace(0.05, 0.1, 10)

# Create an empty list to store function values
function_vector = []

# Loop over alpha and sigma values 
for i, alpha in enumerate(alpha_vals):
    for j, sigma in enumerate(sigma_vals):
        model.par.alpha = alpha
        model.par.sigma = sigma 
        model.solve_wF_vec() 
        model.run_regression()
        function_value = ((0.4-model.sol.beta0)**2 + (-0.1-model.sol.beta1)**2)
        if function_value > 0:
            function_vector.append(function_value)

# Convert the list of function values to a numpy array
function_vals = np.array(function_vector).reshape(len(alpha_vals), len(sigma_vals)) 

# Create a grid of alpha and sigma values 
alpha_grid, sigma_grid = np.meshgrid(alpha_vals, sigma_vals)

# Create plot using plotly
fig = go.Figure(data=[go.Surface(
    x=alpha_grid,
    y=sigma_grid,
    z=function_vals,
    colorscale='plasma',
    hovertemplate='Alpha: %{x:.2f}<br>Sigma: %{y:.2f}<br>Function Value: %{z:.2f}',
)])

# Update layout
fig.update_layout(
    scene=dict(
        xaxis=dict(
            title='alpha',
            tickformat=".2f"
        ),
        yaxis=dict(
            title='sigma',
            tickformat=".2f"
        ),
        zaxis=dict(
            title='Function Value',
            tickformat=".2f"
        )
    ),
    title='Function Values for Different Alpha and Sigma Values'
)

# Show the interactive plot
fig.show()

# Question 5

For this question we add an extension to the given model. We extend the model by adding disutility of household production. This means that the model now takes into account the fact that the household members may not enjoy household production and this disutility may affect their decisions regarding allocating time between work and home production.

$$
\begin{aligned}

disutility = \nu \left( \frac{T_M^{1+\frac{1}{\epsilon}}}{1+ \frac{1}{\epsilon}} + \frac{T_F^{1+\frac{1}{\epsilon}}}{1+ \frac{1}{\epsilon}} \right) + \hat{\nu}H_M + \left(1-\hat{\nu}\right)H_F

\end{aligned}
$$

where

$$
\begin{aligned}

\hat{\nu} \le 1

\end{aligned}
$$

The equation accounts for disutility where $\nu$ is the weight of disutility effect associated with the time spent on market work $(T_M)$ and time spent on home work $(T_F)$.

In [ ]:
# Import inauguralproject for question 5
from Q5 import HouseholdSpecializationModelClassQ5
modelQ5 = HouseholdSpecializationModelClassQ5()

In [ ]:
# Setting model parameters
modelQ5.par.alpha = 0.5
modelQ5.par.sigma = 1.0

# Introducing additional disutility of household labor
modelQ5.par.include_disutil = True

# Defining objective function to minimize difference between model and data
def obj(x):
    modelQ5.par.disutil_HM = x[0]
    modelQ5.par.disutil_HF = x[1]

    modelQ5.solve_wF_vec()
    modelQ5.run_regression()

    # Calculating the difference between beta0 and beta1 for model and target values
    difference = (modelQ5.par.beta0_target - modelQ5.sol.beta0)**2 + (modelQ5.par.beta1_target - modelQ5.sol.beta1)**2

    return difference

# Solving for optimal values of disutility for male and female using Nelder-Mead optimization method
res = optimize.minimize(obj, (0.0315, 0.007), method='Nelder-Mead', bounds=optimize.Bounds([0.0001, 0.0001],[0.05, 0.05], keep_feasible=True), options={})

In [ ]:
print(f'Solution: \n Male disutility = {res.x[0]:.3f} , Female disutility = {res.x[1]:.3f}')

We find that the male disutility of household production is much higher than the female disutility of household production. the different disutility can come from different preferences when it comes to the types of household tasks they prefer to perform. For example, men may prefer tasks that involve outdoor work or using tools, which are not traditionally seen as "feminine" tasks, but which they may find more enjoyable than household cleaning or cooking.

In [ ]:
# Assuming that you have calculated the mean values for each of the variables
original_means = [np.mean(model.sol.LM_vec), np.mean(model.sol.LF_vec), np.mean(model.sol.HM_vec), np.mean(model.sol.HF_vec)]
extended_means = [np.mean(modelQ5.sol.LM_vec), np.mean(modelQ5.sol.LF_vec), np.mean(modelQ5.sol.HM_vec), np.mean(modelQ5.sol.HF_vec)]

# Create a list of labels
labels = ['L_M', 'L_F', 'H_M', 'H_F']

fig = go.Figure(data=[
    go.Bar(name='Original', x=labels, y=original_means, hovertemplate='%{y:.2f}'),
    go.Bar(name='Extended', x=labels, y=extended_means, hovertemplate='%{y:.2f}')
])

# Change the bar mode
fig.update_layout(barmode='group', title_text='Hours Comparison between Original and Extended Model')

fig.show()

By comparing the models we see that both household members are working less hours in the household in the extended model than the original. The number of male hours in household production decreases from 3.4 to 1.0 whereas the female hours decreases from 5.4 to 1.5. The female household member is spending most time at household production. 
By looking at hours spend working, the female hours increases from 3.3 to 6.8 hours and the male working hours increases from 5.3 to 7.4. Here the male is spending most time to work in the market. 

Overall, the extended model places a higher value on market work compared to the original model, while the original model places a higher value on household work. We believe that this model is a better fit than the model from Siminski and Yetsenga (2022) because this model provides a more comprehensive view of decision-making within the household. The disutility can influence the household members' decisions on time allocation between market work and household production.